# Розробка та навчання моделі

In [19]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

__Функція для виведення класифікаційного звіту__ (видалити)

In [20]:
# Видалити
def model_classification_report(model, model_name: str, X_test, y_test):
    y_pred = model.predict(X_test)
    print(
        f"Класифікаційний звіт для моделі {model_name}:\n",
        classification_report(y_test, y_pred),
    )

Зчитування файлу internet_service_churn.csv. (редагувати назву файлу)

In [21]:
df = pd.read_csv("../data/raw/internet_service_churn.csv") # видалити
# df = pd.read_csv("../data/processed/processed_data.csv")  # ??

Перемішаємо дані.

In [22]:
df = shuffle(df, random_state=13)

__Функція для розділення датасету на матриці X і y__

In [23]:
def create_X_y(df):
    X = df[df.columns[1:-1]]
    y = df[df.columns[-1:]].values.flatten()
    return (X, y)

Виділимо матриці X і y.

In [24]:
X, y = create_X_y(df)

Розділимо дані на тренувальні та тестові.

In [25]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=13
)

__Модель Random Forest__

In [26]:
model_RF = RandomForestClassifier(
    n_estimators=150,
    criterion="gini",
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=2,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=True,
    ccp_alpha=0.0,
    max_samples=0.5,
)

Навчання моделі Random Forest.

In [27]:
model_RF.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, max_features=None, max_samples=0.5,
                       min_samples_leaf=2, n_estimators=150, oob_score=True)

In [28]:
# Видалити
model_classification_report(model_RF, "Random Forest", X_test, y_test)

Класифікаційний звіт для моделі Random Forest:
               precision    recall  f1-score   support

           0       0.93      0.95      0.94      6576
           1       0.95      0.94      0.94      7879

    accuracy                           0.94     14455
   macro avg       0.94      0.94      0.94     14455
weighted avg       0.94      0.94      0.94     14455



__Модель Histogram-based Gradient Boosting Classification Tree__

In [29]:
model_HGB = HistGradientBoostingClassifier(
    learning_rate=0.1,
    max_iter=200,
    max_leaf_nodes=21,
    max_depth=None,
    min_samples_leaf=10,
    l2_regularization=10.0,
    max_bins=255,
    interaction_cst=None,
    early_stopping=False,
    scoring=None,
    validation_fraction=0.1,
    n_iter_no_change=5,
    tol=1e-07,
    class_weight=None,
)

Навчання моделі Histogram-based Gradient Boosting.

In [30]:
model_HGB.fit(X_train, y_train)

HistGradientBoostingClassifier(early_stopping=False, l2_regularization=10.0,
                               max_iter=200, max_leaf_nodes=21,
                               min_samples_leaf=10, n_iter_no_change=5,
                               scoring=None)

In [31]:
# Видалити
model_classification_report(
    model_HGB, "Histogram-based Gradient Boosting Classification Tree", X_test, y_test
)

Класифікаційний звіт для моделі Histogram-based Gradient Boosting Classification Tree:
               precision    recall  f1-score   support

           0       0.93      0.95      0.94      6576
           1       0.96      0.94      0.95      7879

    accuracy                           0.94     14455
   macro avg       0.94      0.94      0.94     14455
weighted avg       0.94      0.94      0.94     14455



__Модель Gradient Boosting__

In [32]:
model_GB = GradientBoostingClassifier(
    loss="log_loss",
    learning_rate=0.1,
    n_estimators=150,
    subsample=1.0,
    criterion="friedman_mse",
    min_samples_split=2,
    min_samples_leaf=4,
    min_weight_fraction_leaf=0.0,
    max_depth=5,
    min_impurity_decrease=0.0,
    max_features=None,
)

Навчання моделі Gradient Boosting.

In [33]:
# model_GB.fit(X_train, y_train) # поки непрацює, бо немає стандартизованої таблиці

In [34]:
# Видалити
# model_classification_report(model_GB, "Gradient Boosting", X_test, y_test)

__Модель Logistic Regression__

In [35]:
model_LR = LogisticRegression(penalty=None, solver="newton-cg")

Навчання моделі Logistic Regression.

In [36]:
# model_LR.fit(X_train, y_train)  # поки непрацює, бо немає стандартизованої таблиці

In [37]:
# Видалити
# model_classification_report(model_LR, "Logistic Regression", X_test, y_test)